In [6]:
import boto3
ec2 = boto3.client('ec2')
elb = boto3.client('elbv2')

In [8]:
KEY_NAME = "22962256-key"
SG_NAME = "22962256-sg"
ELB_NAME = "22962256-elb"
TARGET_GROUP_NAME = "22962256-tg"

# Prepare subnet for ec2 creation
VPC_ID = "vpc-00da1b229d10a51b6"
SUBNETS = [
    {"id": "subnet-0a7d8e51199753df1", "name": "ap-southeast-2a"},
    {"id": "subnet-0c1878c6a739707b7", "name": "ap-southeast-2b"},
]

sg_id = "sg-00782b687d5272549"

In [3]:
key_response = ec2.create_key_pair(
    KeyName=KEY_NAME
)

with open("22962256_key.pem", "wb") as file:
    # Write the variable to the file
    file.write(key_response['KeyMaterial'].encode())

In [4]:
# Create Security Group
sg_response = ec2.create_security_group(
    Description="security group for development environment by boto3",
    GroupName=SG_NAME,
    VpcId=VPC_ID,
)

sg_id = sg_response["GroupId"]


ec2.authorize_security_group_ingress(
    GroupId=sg_id,
    IpPermissions=[
        {
            "IpProtocol": "tcp",
            "FromPort": 22,
            "ToPort": 22,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}],
        },
        {
            "IpProtocol": "tcp",
            "FromPort": 80,
            "ToPort": 80,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}],
        },
    ],
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0be190bf4da359e3b',
   'GroupId': 'sg-00782b687d5272549',
   'GroupOwnerId': '489389878001',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '0.0.0.0/0'},
  {'SecurityGroupRuleId': 'sgr-0f5936f55679953ea',
   'GroupId': 'sg-00782b687d5272549',
   'GroupOwnerId': '489389878001',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': 'b15773d4-46b5-4868-a5e0-4bd13fd52402',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b15773d4-46b5-4868-a5e0-4bd13fd52402',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1124',
   'date': 'Thu, 28 Sep 2023 02:43:45 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [9]:
load_balancer_response = elb.create_load_balancer(
    Name=ELB_NAME,
    Subnets=list(map(lambda subnet: subnet["id"], SUBNETS)),
    SecurityGroups=[sg_id],
    Scheme="internet-facing",
    IpAddressType="ipv4",
)

In [12]:
target_group_response = elb.create_target_group(
    Name=TARGET_GROUP_NAME,
    Protocol="HTTP",
    Port=80,
    VpcId=VPC_ID,
    HealthCheckProtocol="HTTP",
    HealthCheckPath="/polls/",
    HealthCheckEnabled=True,
    HealthCheckIntervalSeconds=30,
    HealthCheckTimeoutSeconds=5,
    HealthyThresholdCount=5,
    UnhealthyThresholdCount=2,
    TargetType="instance",
)
target_group_arn = target_group_response["TargetGroups"][0]["TargetGroupArn"]

In [13]:
listener_response = elb.create_listener(
    LoadBalancerArn=load_balancer_response["LoadBalancers"][0]["LoadBalancerArn"],
    Protocol='HTTP',
    Port=80,
    DefaultActions=[
        {
            'Type': 'forward',
            'ForwardConfig': {
                'TargetGroups': [
                    {
                        'TargetGroupArn': target_group_arn,
                    },
                ],
            }
        },
    ],
)

In [14]:
instance_responces = list(
    map(
        lambda subnet: ec2.run_instances(
            # ImageId="ami-0310483fb2b488153",
            ImageId="ami-0c7c2fcf9f0885c8c",
            InstanceType="t2.micro",
            MaxCount=1,
            MinCount=1,
            KeyName=KEY_NAME,   
            SubnetId=subnet["id"],
            SecurityGroupIds=[sg_id],
            # UserData=get_userdata(subnet["name"]),
            TagSpecifications=[
                {
                    "ResourceType": "instance",
                    "Tags": [
                        {"Key": "Name", "Value": "22962256-" + subnet["name"]},
                    ],
                },
            ],
        ),
        SUBNETS,
    )
)

In [15]:
instance_ids = list(
    map(lambda response: response["Instances"][0]["InstanceId"], instance_responces)
)
register_targets_response = elb.register_targets(
    TargetGroupArn=target_group_arn,
    Targets=list(map(lambda id: {"Id": id, "Port": 80}, instance_ids)),
)